In [1]:
from gezi.common import *
sys.path.append('..')
from src.config import *
from src.preprocess import *
from src.eval import *
gezi.init_flags()

In [2]:
root = '../working/offline/6/0'
# context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.p9'
# pairwise_model_name = 'deberta-v3-small.flag-pairwise9'
context_model_name = 'deberta-v3-small.flag-context2-aug.n_context-40.cls_loss_rate-0.1.eval.mpnet13-2'
pairwise_model_name = 'all-mpnet-base-v2.flag-pairwise13-2'

In [3]:
xc = gezi.load(f'{root}/{context_model_name}/valid.pkl')

In [4]:
xp = gezi.load(f'{root}/{pairwise_model_name}/valid.pkl')

In [5]:
gezi.sort_dict_byid_(xc, 'cid')
gezi.sort_dict_byid_(xp, 'cid')

In [6]:
ids = set(xc['id'])

In [7]:
df_gt = pd.read_csv(f'{FLAGS.root}/train_orders.csv')
df_gt = df_gt[df_gt.id.isin(ids)]
df_gt['cell_order'] = df_gt['cell_order'].apply(lambda x: x.split())
df_gt.head()

,id,cell_order
4,0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe..."
11,00062ab8487156,"[dcad687f, a2e1fc80, 7d977ee8, 45a82a59, cbbc3..."
28,000efd285fb982,"[74a30f80, ee2c8e08, 5523374e, ae8f8fe8, 2138e..."
39,0012865b766949,"[f9cb50e9, 25f7db90, d804e819, 6593a545, fc5bb..."
42,001308991e0c5e,"[21147235, 6c01d0d2, 5bd28595, b8fd3a8c, a2501..."


In [8]:
calc_metric(xc, 'reg_pred')

0.9008516068000336

In [9]:
calc_metric(xc, 'pred')

0.8995007840615565

In [10]:
calc_metric(xc, 'cls_pred')

0.8963390358922603

In [11]:
calc_metric(xp, 'cls_pred')

0.8918236725011705

In [12]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.9025047569234644

In [28]:
x = xc.copy()
x['pred'] = xc['cls_pred'] * 0.5 + xp['cls_pred'] * 0.5
calc_metric(x)

0.8998311184115386

In [29]:
x = xc.copy()
x['pred'] = xc['reg_pred'] * 0.5 + xp['pred'] * 0.5
calc_metric(x)

0.9014206984735855

In [13]:
def merge(x, y, prob):
  # return y
  # return x
  if prob > 0.9:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.1:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.2:
    return x * 0.95 * prob + y * (1 - 0.95 * prob)
  elif abs(y - x) < 0.3:
    return x * 0.85 * prob + y * (1 - 0.85 * prob)
  elif abs(y - x) < 0.4:
    return x * 0.5 * prob + y * (1 - 0.5 * prob)
  else:
    return y

In [14]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['pred'], xp['max_prob'])]
calc_metric(x)

0.9073869212113768

In [15]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['reg_pred'], xp['max_prob'])]
calc_metric(x)

0.9070471416972357

In [16]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['pred'], xc['cls_pred'], xp['max_prob'])]
calc_metric(x)

0.9068742541271928

In [17]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['cls_pred'], xc['cls_pred'], xp['max_prob'])]
calc_metric(x)

0.9007170337437319

In [18]:
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(xp['cls_pred'], xc['reg_pred'], xp['max_prob'])]
calc_metric(x)

0.9012548742436964

In [19]:
df_p = pd.DataFrame(xp)

In [20]:
df_c = pd.DataFrame(gezi.batch2list(xc))

In [23]:
df_p.columns

Index(['id', 'cell_id', 'cid', 'pred', 'max_prob', 'max_sim', 'cls_pred',
       'match_rank', 'match_code', 'n_code_cell', 'probs', 'sims'],
      dtype='object')

In [24]:
df = df_c.merge(df_p[['cid', 'pred', 'cls_pred', 'max_prob', 'max_sim', 'probs', 'sims']], on='cid', suffixes=['_c', '_p'])

In [26]:
df_train = pd.read_feather('../working/train.fea')

In [27]:
df_train = df_train[df_train.id.isin(ids)]

In [30]:
df = df.merge(df_train[['cid', 'ancestor_id', 'n_words', 'source']], on='cid')

In [31]:
gezi.set_fold(df, 5, 'ancestor_id')

In [32]:
df.head()

,id,cell_id,cid,cell_type,rank,rel_rank,context_match,match_code,n_code_cell,n_markdown_cell,...,pred_p,cls_pred_p,max_prob,max_sim,probs,sims,ancestor_id,n_words,source,fold
0,0002115f48f982,9ec225f0,0002115f48f982\t9ec225f0,markdown,0,0.055556,1,0.0,8,1,...,0.055556,0.071522,0.875342,0.817135,"[0.8753421, 0.11626246, 0.0010800936, 0.005473...","[0.81713474, 0.7799119, 0.6936422, 0.7235639, ...",272b483a,49,"Hi there,\n\nIs it ok that the same measuremen...",2
1,00062ab8487156,96c8449c,00062ab8487156\t96c8449c,markdown,11,0.954545,1,10.0,10,2,...,0.954545,0.949661,0.976317,0.838338,"[0.0026414075, 0.00013251408, 0.00015031976, 1...","[0.729321, 0.6741463, 0.6764709, 0.62993234, 0...",de362dd8,9,### Support vector classifier scored 0.8442 on...,1
2,00062ab8487156,aa354742,00062ab8487156\taa354742,markdown,5,0.500000,1,5.0,10,2,...,0.500000,0.461070,0.701096,0.729470,"[0.00034627708, 0.0022056992, 0.17445873, 0.01...","[0.5890951, 0.623235, 0.7038227, 0.6558083, 0....",de362dd8,40,### Interesting tidbit: Shovel Knight is the o...,1
3,000efd285fb982,14f36391,000efd285fb982\t14f36391,markdown,56,0.881579,1,33.0,37,28,...,0.881579,0.881693,0.995167,0.786112,"[2.9696256e-08, 2.2839872e-10, 1.4825383e-10, ...","[0.46662188, 0.37686953, 0.36890104, 0.3841684...",389f14ff,3,### 3.5 CatboostRegressor,4
4,000efd285fb982,1f3749ad,000efd285fb982\t1f3749ad,markdown,24,0.335526,1,12.0,37,28,...,0.328947,0.328956,0.996717,0.752300,"[7.0347337e-06, 2.974339e-06, 4.029498e-05, 3....","[0.53359485, 0.51772237, 0.56577665, 0.5614152...",389f14ff,4,### 2.1 Numerical columns,4


In [92]:
df['pred_diff0'] = abs(df['pred_c'] - df['pred_p'])
df['pred_diff1'] = abs(df['reg_pred'] - df['pred_p'])
df['pred_diff2'] = abs(df['cls_pred_c'] - df['pred_p'])
df['pred_diff3'] = abs(df['cls2_pred'] - df['pred_p'])
df['markdown_frac'] = df['n_markdown_cell'] / df['n_cell']
df['span'] = 1 / (df['n_code_cell'] + 1)
top2, top3, top4, top5 = [], [], [], []
top2_prob, top3_prob, top4_prob, top5_prob = [], [], [], []
top2_sim, top3_sim, top4_sim, top5_sim = [], [], [], []
for i in tqdm(range(len(df)), desc='top'):
  # cls_preds = df['cls_pred_ori'].values[i]
  n_code = df['n_code_cell'].values[i]
  probs = df['probs'].values[i]
  sims = df['sims'].values[i]
  idxes = (-probs).argsort()
  if len(idxes) > 1:
    top2.append((idxes[1] + 0.5) / (n_code + 1))
    top2_prob.append(probs[idxes[1]])
    top2_sim.append(sims[idxes[1]])
  else:
    top2.append(-1)
    top2_prob.append(-1)
    top2_sim.append(-1)
  if len(idxes) > 2:
    top3.append((idxes[2] + 0.5) / (n_code + 1))
    top3_prob.append(probs[idxes[2]])
    top3_sim.append(sims[idxes[2]])
  else:
    top3.append(-1)
    top3_prob.append(-1)
    top3_sim.append(-1)
  if len(idxes) > 3:
    top4.append((idxes[3] + 0.5) / (n_code + 1))
    top4_prob.append(probs[idxes[3]])
    top4_sim.append(sims[idxes[3]])
  else:
    top4.append(-1)
    top4_prob.append(-1)
    top4_sim.append(-1)
  if len(idxes) > 4:
    top5.append((idxes[4] + 0.5) / (n_code + 1))
    top5_prob.append(probs[idxes[4]])
    top5_sim.append(sims[idxes[4]])
  else:
    top5.append(-1)
    top5_prob.append(-1)
    top5_sim.append(-1)
ctop_prob, ctop2, ctop3, ctop4, ctop2_prob, ctop3_prob, ctop4_prob = [], [], [], [], [], [], []
for i in tqdm(range(len(df)), desc='ctop'):
  preds = df['cls_pred_ori'].values[i]
  probs = gezi.softmax(preds)
  idxes = (-probs).argsort()
  ctop_prob.append(probs[idxes[0]])
  ctop2.append((idxes[1] + 0.5) / FLAGS.num_classes)
  ctop2_prob.append(probs[idxes[1]])
  ctop3.append((idxes[2] + 0.5) / FLAGS.num_classes)
  ctop3_prob.append(probs[idxes[2]])
  ctop4.append((idxes[3] + 0.5) / FLAGS.num_classes)
  ctop4_prob.append(probs[idxes[3]])
# for i in range(FLAGS.num_classes):
#   df[f'cls_pred{i}'] = df['cls_pred_ori'].apply(lambda x: x[i])
df['top2'] = top2
df['top2_prob'] = top2_prob
df['top2_sim'] = top2_sim
df['top3'] = top3
df['top3_prob'] = top3_prob
df['top3_sim'] = top3_sim
df['top4'] = top4
df['top4_prob'] = top4_prob
df['top4_sim'] = top4_sim
df['top5'] = top5
df['top5_prob'] = top5_prob
df['top5_sim'] = top5_sim
df['ctop_prob'] = ctop_prob
df['ctop2'] = ctop2
df['ctop2_prob'] = ctop2_prob
df['ctop3'] = ctop3
df['ctop3_prob'] = ctop3_prob
df['ctop4'] = ctop4
df['ctop4_prob'] = ctop4_prob
df['pred_diff4'] = abs(df['pred_c'] - df['top2'])
df['pred_diff5'] = abs(df['reg_pred'] - df['top2'])
df['pred_diff6'] = abs(df['cls_pred_c'] - df['top2'])
df['pred_diff7'] = abs(df['cls2_pred'] - df['top2'])
df['pred_diff8'] = abs(df['pred_p'] - df['top2'])
df['pred_diff9'] = abs(df['cls_pred_c'] - df['top3'])
df['pred_diff10'] = abs(df['pred_p'] - df['top3'])
df['rule_pred'] = [merge(x, y, prob) for x, y, prob in tqdm(zip(df.pred_p.values, df.pred_c.values, df.max_prob.values), total=len(df), desc='rule')]
reg_pred_probs = []
for row in tqdm(df.itertuples(), total=len(df), desc='cls_pred_p_prob'):
  probs = row.probs
  n_code = row.n_code_cell
  reg_pred = row.reg_pred
  try:
    reg_pred_probs.append(probs[int(min(reg_pred, 1) * (n_code + 1) - 0.5)])
  except Exception:
    ic(n_code, len(probs), reg_pred)
    break
df['reg_pred_prob'] = reg_pred_probs

top:   0%|          | 0/424943 [00:00<?, ?it/s]

ctop:   0%|          | 0/424943 [00:00<?, ?it/s]

rule:   0%|          | 0/424943 [00:00<?, ?it/s]

cls_pred_p_prob:   0%|          | 0/424943 [00:00<?, ?it/s]

In [111]:
fold = 0
dvalid = df[df.fold==fold]
dtrain = df[df.fold!=fold]

In [112]:
df.columns

Index(['id', 'cell_id', 'cid', 'cell_type', 'rank', 'rel_rank',
       'context_match', 'match_code', 'n_code_cell', 'n_markdown_cell',
       'n_cell', 'cls_pred_c', 'pred_c', 'reg_pred', 'cls_pred_ori',
       'match_rank', 'cls2_pred', 'pred_p', 'cls_pred_p', 'max_prob',
       'max_sim', 'probs', 'sims', 'ancestor_id', 'n_words', 'source', 'fold',
       'pred_diff0', 'pred_diff1', 'pred_diff2', 'pred_diff3', 'markdown_frac',
       'span', 'top2', 'top2_prob', 'top2_sim', 'top3', 'top3_prob',
       'top3_sim', 'top4', 'top4_prob', 'top4_sim', 'top5', 'top5_prob',
       'top5_sim', 'ctop_prob', 'ctop2', 'ctop2_prob', 'ctop3', 'ctop3_prob',
       'ctop4', 'ctop4_prob', 'pred_diff4', 'pred_diff5', 'pred_diff6',
       'pred_diff7', 'pred_diff8', 'pred_diff9', 'pred_diff10', 'rule_pred',
       'reg_pred_prob'],
      dtype='object')

In [113]:
df

,id,cell_id,cid,cell_type,rank,rel_rank,context_match,match_code,n_code_cell,n_markdown_cell,...,ctop4_prob,pred_diff4,pred_diff5,pred_diff6,pred_diff7,pred_diff8,pred_diff9,pred_diff10,rule_pred,reg_pred_prob
0,0002115f48f982,9ec225f0,0002115f48f982\t9ec225f0,markdown,0,0.055556,1,0.0,8,1,...,0.001837,0.106798,0.107313,0.106283,0.111667,0.111111,0.328505,0.333333,0.055556,8.753421e-01
1,00062ab8487156,96c8449c,00062ab8487156\t96c8449c,markdown,11,0.954545,1,10.0,10,2,...,0.011861,0.083150,0.098536,0.067765,0.091364,0.090909,0.885947,0.909091,0.954545,9.763172e-01
2,00062ab8487156,aa354742,00062ab8487156\taa354742,markdown,5,0.500000,1,5.0,10,2,...,0.047884,0.237985,0.249040,0.226929,0.277727,0.272727,0.136707,0.090909,0.499997,2.804984e-02
3,000efd285fb982,14f36391,000efd285fb982\t14f36391,markdown,56,0.881579,1,33.0,37,28,...,0.051794,0.044795,0.040921,0.048669,0.022895,0.026316,0.074985,0.052632,0.881577,8.346825e-08
4,000efd285fb982,1f3749ad,000efd285fb982\t1f3749ad,markdown,24,0.335526,1,12.0,37,28,...,0.037001,0.019517,0.020190,0.018843,0.020263,0.026316,0.060104,0.052632,0.328948,9.967172e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424938,fff4714a37cf49,9527a079,fff4714a37cf49\t9527a079,markdown,10,0.500000,1,6.0,12,9,...,0.035817,0.253585,0.250999,0.256171,0.225769,0.230769,0.051521,0.076923,0.499998,6.165756e-02
424939,fff4714a37cf49,effc5f17,fff4714a37cf49\teffc5f17,markdown,6,0.346154,1,4.0,12,9,...,0.038901,0.022014,0.001136,0.045164,0.001154,0.076923,0.185605,0.307692,0.269241,4.972901e-01
424940,fff4714a37cf49,f8744de1,fff4714a37cf49\tf8744de1,markdown,17,0.807692,1,10.0,12,9,...,0.025195,0.314488,0.319733,0.309243,0.305000,0.307692,0.232320,0.230769,0.807693,9.271661e-01
424941,fffc1b4f08e81c,143cccfb,fffc1b4f08e81c\t143cccfb,markdown,29,0.730769,1,28.0,38,2,...,0.059311,0.111172,0.114996,0.107348,0.209615,0.282051,0.209912,0.384615,0.674745,2.885038e-03


In [135]:
reg_cols =  [
          'n_code_cell',
          'n_markdown_cell',
          'n_cell',
          'cls_pred_c',
          'pred_c',
          'reg_pred',
          'cls2_pred',
          'pred_p',
          'cls_pred_p',
          'rule_pred',
          'pred_diff0',
          'pred_diff1',
          'pred_diff2',
          'pred_diff3',
          'pred_diff4',
          'pred_diff5',
          'pred_diff6',
          'pred_diff7',
          'pred_diff8',
          'pred_diff9',
          'pred_diff10',
          'max_sim',
          'max_prob',
          'markdown_frac',
          'span',
          'top2',
          'top2_prob',
          'top2_sim',
          'top3',
          'top3_prob',
          'top3_sim',
          'top4',
          'top4_prob',
          'top4_sim',
          'top5',
          # 'top5_prob',
          # 'top5_sim',
          'ctop_prob', 
          'ctop2', 
          # 'ctop3', 
          # 'ctop4', 
          'ctop2_prob', 
          # 'ctop3_prob', 
          # 'ctop4_prob',
          # 'cls_pred_ori',
          # 'cls2_pred_top2',
          # 'cls2_pred_top3',
          # 'sims',
          # 'probs',
          # 'cls_pred_p_prob',
          # 'reg_pred_prob',
        ]
# for i in range(FLAGS.num_classes):
#   reg_cols += [f'cls_pred{i}']
cat_cols = [
          
            ]
label_col = 'rel_rank'
cols = reg_cols + cat_cols
X_train = dtrain[cols]
y_train = dtrain[label_col]
X_valid = dvalid[cols]
y_valid = dvalid[label_col]

In [136]:
params = {
          'boosting': 'gbdt',
          'objective': 'regression_l1',
          'metric': {'l1'},
          'num_leaves': 8,
          'min_data_in_leaf': 30,
          'max_depth': 6,
          'learning_rate': 0.01,
          "feature_fraction": 0.9,
          "bagging_fraction": 0.75,
          'min_data_in_bin':15,
          "lambda_l1": 5,
          'lambda_l2': 5,
          "random_state": 1024,
          "num_threads": 12,
          }

In [137]:
# import lightgbm as lgb
# d_train = lgb.Dataset(X_train, y_train)
# d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)

# bst = lgb.train(params,
#                 d_train,
#                 10000,
#                 valid_sets=[d_train, d_valid],
#                 verbose_eval=10,
#                 early_stopping_rounds=100)

In [138]:
# abs(dvalid['pred_c'] - dvalid['rel_rank']).mean()

In [139]:
# dvalid['lgb_pred'] = bst.predict(dvalid[cols])

In [140]:
# calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.lgb_pred.values})

In [141]:
from catboost import CatBoostRegressor, Pool
xgb_params = {'learning_rate': 0.02,
              'reg_lambda': 7.960622217848342e-07, 
              'subsample': 0.7422597612762745,
              'max_depth': 10, 
              'early_stopping_rounds': 500,
              'n_estimators': 10000,
              'cat_features': [],
              'loss_function': 'MAE',
              }

xgb_params2 = {'learning_rate': 0.09827605967564293,'tree_method':'gpu_hist', 'gpu_id':0,
               'early_stopping_rounds': 50,
               'n_estimators': 10000, }

In [142]:
model = CatBoostRegressor(**xgb_params)
model.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
                verbose=10,
                )  

0:	learn: 0.2544977	test: 0.2544977	test1: 0.2544289	best: 0.2544289 (0)	total: 53.4ms	remaining: 8m 54s
10:	learn: 0.2165309	test: 0.2165309	test1: 0.2167191	best: 0.2167191 (10)	total: 558ms	remaining: 8m 27s
20:	learn: 0.1860034	test: 0.1860034	test1: 0.1864271	best: 0.1864271 (20)	total: 1.03s	remaining: 8m 9s
30:	learn: 0.1614847	test: 0.1614847	test1: 0.1621053	best: 0.1621053 (30)	total: 1.48s	remaining: 7m 57s
40:	learn: 0.1419327	test: 0.1419327	test1: 0.1427275	best: 0.1427275 (40)	total: 1.96s	remaining: 7m 55s
50:	learn: 0.1262520	test: 0.1262520	test1: 0.1271952	best: 0.1271952 (50)	total: 2.42s	remaining: 7m 53s
60:	learn: 0.1137402	test: 0.1137402	test1: 0.1148146	best: 0.1148146 (60)	total: 2.9s	remaining: 7m 52s
70:	learn: 0.1037060	test: 0.1037060	test1: 0.1049168	best: 0.1049168 (70)	total: 3.35s	remaining: 7m 48s
80:	learn: 0.0957690	test: 0.0957690	test1: 0.0971251	best: 0.0971251 (80)	total: 3.82s	remaining: 7m 47s
90:	learn: 0.0894936	test: 0.0894936	test1: 0.090

In [143]:
# model = CatBoostRegressor()
# model.load_model('../working/cbt/cbt.bin')

In [144]:
dvalid['cb_pred'] = model.predict(dvalid[cols])

In [145]:
x = {'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values}
x['pred'] = [merge(x, y, prob) for x, y, prob in zip(dvalid.pred_p.values, dvalid.pred_c.values, dvalid.max_prob.values)]
score = calc_metric(x, 'pred', df_gt)
ic(score)
ic(abs(x['pred'] - dvalid['rel_rank']).mean())

[07/18/22 11:10:38] 599230365.py:4 in <module>
                    score: 0.9062897096385878
[07/18/22 11:10:38] 599230365.py:5 in <module>
                    abs(x['pred'] - dvalid['rel_rank']).mean(): 0.0666019292599528


0.0666019292599528

In [146]:
ic(calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}))

[07/18/22 11:10:45] 2876217204.py:1 in <module>
                    calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.cb_pred.values}): 0.9083603315872429


0.9083603315872429

In [147]:
model.save_model('../working/cbt.bin')

In [148]:
gezi.plot.feature_importance(model, topn=20)

In [149]:
gezi.plot.feature_importance(model, topn=-20)

In [128]:
stop

NameError: name 'stop' is not defined

In [ ]:
dvalid[cols + ['cb_pred']]

In [ ]:
calc_metric({'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values, 'pred': dvalid.reg_pred.values})

In [ ]:
import optuna
from optuna import Trial

In [ ]:
P = {}
def merge(x, y, prob):
  if prob > P['prob']:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < P['diff']:
    return x * (1 - 0.0001) + y * 0.0001
  elif abs(y - x) < 0.2:
    return x * P['rate0'] * prob + y * (1 - P['rate0'] * prob)
  elif abs(y - x) < 0.3:
    return x * P['rate1'] * prob + y * (1 - P['rate1'] * prob)
  elif abs(y - x) < 0.4:
    return x * P['rate2'] * prob + y * (1 - P['rate2'] * prob)
  elif abs(y - x) < 0.5:
    return x * P['rate3'] * prob + y * (1 - P['rate3'] * prob)
  else:
    return x * P['rate4'] * prob + y * (1 - P['rate4'] * prob)

In [ ]:
df_gt = df_gt[df_gt.id.isin(ids)]
x = {'id': dtrain.id.values, 'cell_id': dtrain.cell_id.values}

In [ ]:
# def objective(trial):
#   suggest = trial.suggest_float 
#   P['prob'] = suggest('prob', 0., 1.)
#   P['diff'] = suggest('diff', 0., 1.)
#   P['rate0'] = suggest('rate0', 0., 1.)
#   P['rate1'] = suggest('rate1', 0., 1.)
#   P['rate2'] = suggest('rate2', 0., 1.)
#   P['rate3'] = suggest('rate3', 0., 1.)
#   P['rate4'] = suggest('rate4', 0., 1.)
          
#   x['pred'] = [merge(x, y, prob) for x, y, prob in zip(dtrain.pred_p.values, dtrain.pred_c.values, dtrain.max_prob.values)]
#   score = calc_metric(x, 'pred', df_gt)
#   return score
  
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)
# ic(study.best_value, study.best_params)

In [ ]:
# x = {'id': dvalid.id.values, 'cell_id': dvalid.cell_id.values}
# x['pred'] = [merge(x, y, prob) for x, y, prob in zip(dvalid.pred_p.values, dvalid.pred_c.values, dvalid.max_prob.values)]
# score = calc_metric(x, 'pred', df_gt)
# ic(score)